In [2]:
!pip install scikit-learn
!pip install shap
!pip install umap
!pip install umap-learn

In [1]:
import polars as pl
import gzip
from sklearn.cluster import DBSCAN
from sklearn.ensemble import  RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np
from umap import UMAP

/home/marialejandramantillab/anaconda3/envs/dmef/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pl.read_csv('~/buckets/b1/datasets/competencia_02_targets_b2.csv.gz', infer_schema_length=100000, dtypes={'foto_mes':str})

In [4]:
df.filter(pl.col("numero_de_cliente") == 29199686)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,…,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
i64,str,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,f64,i64,…,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,str
29199686,"""201901""",1,0,1,51,296,815.72,16932.85,1074.42,-223.35,2.81,7,1,1,0.0,-66.52,2,210.54,0.0,0.0,1,-1031.14,1,0,0.0,1,5,2898.67,1,2,1826.17,0,0.0,0,0.0,0,…,-492.3,0.0,230318.55,0.0,0.0,8,0.0,-766.02,0.0,6633,-492.3,1,0,105.57,0,0,207286.69,-1033,null,2622.48,3032.29,43.87,2266.49,15.75,230318.55,0.0,0.0,8,0.0,-2218.82,1.18,7389,2266.49,8,0,281.52,"""CONTINUA"""
29199686,"""201902""",1,0,1,51,297,801.02,16277.01,987.64,-209.39,50.3,7,1,1,0.0,-528.34,2,967.88,0.0,0.0,1,864.32,1,0,0.0,1,3,2742.01,1,3,8401.4,0,0.0,0,0.0,0,…,6575.25,0.0,243435.62,0.0,0.0,8,0.0,-1354.81,0.0,6661,6575.25,1,0,469.2,0,0,219092.06,-1005,null,2400.5,2769.41,46.36,2619.75,1.94,243435.62,0.0,0.0,8,0.0,-3079.12,0.0,7417,2619.75,4,0,175.95,"""CONTINUA"""
29199686,"""201903""",1,0,1,51,298,1217.08,15391.29,1331.17,-237.92,140.65,7,1,1,0.0,-104.97,2,704.1,0.0,0.0,1,-813.6,1,0,0.0,1,5,7704.17,1,2,1826.14,0,0.0,0,0.0,0,…,0.0,0.0,252796.16,0.0,0.0,11,1847.47,-8498.38,0.0,6692,0.0,0,0,129.03,0,0,227516.54,-974,null,6702.39,7813.75,48.15,7313.39,8.29,252796.16,0.0,0.0,11,7870.83,-2838.83,1.0,7448,7313.39,7,0,457.47,"""CONTINUA"""
29199686,"""201904""",1,0,0,51,299,1428.23,14937.19,1286.83,-49.98,170.52,7,1,1,0.0,-967.65,2,0.0,0.0,0.0,1,-2469.75,1,0,0.0,1,5,2906.77,1,1,732.13,0,0.0,0,0.0,0,…,0.0,0.0,788256.0,0.0,0.0,6,0.0,-1847.47,0.0,6722,0.0,0,0,70.38,0,0,709430.4,-944,null,2628.23,3030.24,52.66,2216.81,14.22,788256.0,0.0,0.0,6,0.0,-7870.83,1.0,7478,2216.81,8,0,281.52,"""CONTINUA"""
29199686,"""201905""",1,0,1,51,300,0.0,0.0,0.0,0.0,0.0,7,1,1,0.0,-2674.15,2,0.0,0.0,0.0,1,-4191.22,1,0,0.0,1,4,3059.67,1,1,2460.35,0,0.0,0,0.0,0,…,2460.35,0.0,788256.0,0.0,0.0,9,0.0,-741.33,0.0,6753,2460.35,1,0,117.3,0,0,709430.4,-913,null,2720.65,3138.43,52.89,2680.7,7.24,788256.0,0.0,0.0,9,0.0,-3088.5,0.0,7509,2680.7,6,0,246.33,"""CONTINUA"""
29199686,"""201906""",1,0,1,52,301,1208.97,14500.06,1282.18,-158.5,96.09,7,1,1,0.0,-4611.42,2,0.0,0.0,0.0,1,-6049.2,1,0,0.0,1,8,5620.94,1,0,0.0,0,0.0,0,0.0,0,…,0.0,0.0,788256.0,0.0,0.0,12,99.7,-2492.62,0.0,6783,0.0,0,0,93.84,0,0,709430.4,-883,null,4895.89,5692.08,50.79,5508.67,1.45,788256.0,0.0,0.0,12,5759.43,-3202.29,0.0,7539,5508.67,9,0,340.17,"""CONTINUA"""
29199686,"""201907""",1,0,1,52,302,1053.22,13554.05,1384.91,-287.73,4.96,7,1,1,0.0,-6429.94,2,0.0,0.0,0.0,1,-8047.71,1,0,0.0,1,

In [5]:
df.unique(subset=["numero_de_cliente"]).shape

(18967, 155)

In [6]:
max_lag = 12
columns = df.columns 
columns.remove('foto_mes')
columns.remove('numero_de_cliente')
columns.remove('clase_ternaria')

for column in columns:
    for lag in range(1, max_lag+1):    
        column_lag = column + '_lag_' + str(lag)
        df = df.with_columns([pl.col(column).shift(lag).over('numero_de_cliente').alias(column_lag)])

In [21]:
df = df.filter((pl.col('clase_ternaria').ne('CONTINUA') | (pl.col('clase_ternaria').eq('CONTINUA') & pl.lit(np.random.rand(df.height)).lt(0.1))))

In [22]:
list([c for c in df.columns if not 'lag' in c])

['numero_de_cliente',
 'foto_mes',
 'active_quarter',
 'cliente_vip',
 'internet',
 'cliente_edad',
 'cliente_antiguedad',
 'mrentabilidad',
 'mrentabilidad_annual',
 'mcomisiones',
 'mactivos_margen',
 'mpasivos_margen',
 'cproductos',
 'tcuentas',
 'ccuenta_corriente',
 'mcuenta_corriente_adicional',
 'mcuenta_corriente',
 'ccaja_ahorro',
 'mcaja_ahorro',
 'mcaja_ahorro_adicional',
 'mcaja_ahorro_dolares',
 'cdescubierto_preacordado',
 'mcuentas_saldo',
 'ctarjeta_debito',
 'ctarjeta_debito_transacciones',
 'mautoservicio',
 'ctarjeta_visa',
 'ctarjeta_visa_transacciones',
 'mtarjeta_visa_consumo',
 'ctarjeta_master',
 'ctarjeta_master_transacciones',
 'mtarjeta_master_consumo',
 'cprestamos_personales',
 'mprestamos_personales',
 'cprestamos_prendarios',
 'mprestamos_prendarios',
 'cprestamos_hipotecarios',
 'mprestamos_hipotecarios',
 'cplazo_fijo',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'cinversion1',
 'minversion1_pesos',
 'minversion1_dolares',
 'cinversion2',
 'minversi

In [23]:
df.shape

(87628, 1979)

In [10]:
df_solo_baja = df.filter(pl.col('clase_ternaria') == 'BAJA+2')

In [11]:
df_solo_baja.shape

(18987, 1979)

In [12]:
def distanceMatrix(model, X):

    terminals = model.apply(X)
    nTrees = terminals.shape[1]

    a = terminals[:,0]
    proxMat = 1*np.equal.outer(a, a)

    for i in range(1, nTrees):
        a = terminals[:,i]
        proxMat += 1*np.equal.outer(a, a)

    proxMat = proxMat / nTrees

    return proxMat.max() - proxMat


imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')

XRF = df.drop(['clase_ternaria'])
YRF = df['clase_ternaria']

XRF_imp = imp_mean.fit_transform(XRF)

model = RandomForestClassifier(n_estimators=100, max_features=20, min_samples_leaf=400, )
model.fit(XRF_imp, YRF)
d = distanceMatrix(model, XRF_imp)

In [19]:
embedding_rf = UMAP(
  n_components=2,
  n_neighbors=50,
  metric="precomputed",
).fit_transform(d[(YRF=="BAJA+2").index,][:,YRF=="BAJA+2"])

plt.scatter(embedding_rf[:,0], embedding_rf[:,1])

AttributeError: 'Series' object has no attribute 'index'

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])